In [2]:
## use conda create -n geo_spark apache-sedona[spark] jupyter_notebook
import os
# Set Hadoop environment variables BEFORE importing Spark
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['PATH'] + r';C:\hadoop\bin'
# os.environ['JAVA_HOME'] = r'path_to_your_java_installation'  # Make sure this points to your Java installation

from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator

# # Initialize Spark Session
# spark = SparkSession.builder \
#     .appName("Sedona_Local") \
#     .master("local[*]") \
#     .config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.0,org.datasyslab:geotools-wrapper:1.4.0-28.2") \
#     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#     .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
#     .getOrCreate()
spark = SparkSession.builder \
    .appName("Sedona_Local") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.0,org.datasyslab:geotools-wrapper:1.4.0-28.2") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .getOrCreate()

# Register Sedona functions
SedonaRegistrator.registerAll(spark)
# Test Sedona
df = spark.sql("SELECT ST_Point(1.0, 2.0) AS point")
df.show(truncate=False)

+-----------+
|point      |
+-----------+
|POINT (1 2)|
+-----------+



C:\Users\es422\AppData\Local\Temp\ipykernel_32192\739727030.py:30: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


In [3]:
from sedona.sql import ST_GeomFromWKT
# Load the Parquet file
df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\Case_03032025_1240.parquet")

# Filter CASE_STATUS_nlsdb != 'Done'
filtered_df = df.filter(df.CSE_DISP != "Closed")
# filtered_df.count()
# Convert WKT/WKB/GeoJSON to Sedona Geometry (Assuming you have a WKT column named 'geometry')
filtered_df = filtered_df.withColumn("geom", ST_GeomFromWKT(df["Shape"]))

# Register DataFrame as a Temp Table for SQL Queries
filtered_df.createOrReplaceTempView("polygons")

# Find Overlapping Polygons using Sedona SQL
overlapping_polygons = spark.sql("""
    SELECT a.*
    FROM polygons a
    JOIN polygons b
    ON ST_Overlaps(a.geom, b.geom)
    WHERE a.id != b.id
""")

# Show Results
overlapping_polygons.count()

Py4JJavaError: An error occurred while calling o69.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 17 in stage 9.0 failed 1 times, most recent failure: Lost task 17.0 in stage 9.0 (TID 24) (laptop executor driver): org.locationtech.jts.io.ParseException: Unknown geometry type: � (line 2)
	at org.locationtech.jts.io.WKTReader.parseErrorWithLine(WKTReader.java:673)
	at org.locationtech.jts.io.WKTReader.readGeometryTaggedText(WKTReader.java:765)
	at org.locationtech.jts.io.WKTReader.readGeometryTaggedText(WKTReader.java:718)
	at org.locationtech.jts.io.WKTReader.read(WKTReader.java:258)
	at org.locationtech.jts.io.WKTReader.read(WKTReader.java:239)
	at org.apache.sedona.common.Constructors.geomFromWKT(Constructors.java:30)
	at org.apache.spark.sql.sedona_sql.expressions.ST_GeomFromWKT$$anonfun$$lessinit$greater$1.apply(Constructors.scala:184)
	at org.apache.spark.sql.sedona_sql.expressions.ST_GeomFromWKT$$anonfun$$lessinit$greater$1.apply(Construct